# Ch3.Retrieval
+ 1. 미지의 데이터를 처리하려면
+ 2. 주어진 PDF를 기반으로 답변하는 챗봇만들기
+ 3. RetrivalQA
+ 4. 준비된 Retrievals를 사용하여 위키백과를 활용

## 1.미지의 데이터를 처리하려면
> Retrieval은 모델이 학습하지 않은 개념도 제공하며, 정보처리를 지원하는 모듈
- 공개되지 않은 기업의 내용은 chatgpt등 학습 못했기에 답변 어려움 
    - RAG(Retriever-Augmented Generation)을 구축, 활용하여 답변 가능해짐
- RAG는 어떻게 작동하나?
    1. 받은 질문에 관련하여, DATA에서 필요한 문장 찾기
    2. 질문과 찾은 문장 조합하여 프롬프트 생성
    3. 생성된 프롬프트를 언어모델에 전달
- 받은 질문 벡터화 → (미리 벡터화된 )Data에서 유사도 높은 문장 찾음

In [3]:
'''
OpenAIEmbeddings을 활용하여, 질문과 답변을 임베딩하고, 연관성을 확인해보자
'''

from langchain.embeddings import OpenAIEmbeddings  #← OpenAIEmbeddings를 가져오기
from numpy import dot  #← 벡터의 유사도를 계산하기 위해 dot을 가져오기
from numpy.linalg import norm  #← 벡터의 유사도를 계산하기 위해 norm을 가져오기
import warnings
from dotenv import load_dotenv
import openai
import os
warnings.filterwarnings('ignore')
load_dotenv()
os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings( #← OpenAIEmbeddings를 초기화
    model="text-embedding-ada-002"
)
question = "비행 자동차의 최고 속도는?"
doc1 = "비행 자동차의 최고 속도는 시속 150km입니다."
doc2 = "닭고기를 적당히 양념한 후 중불로 굽다가 가끔 뒤집어 주면서 겉은 고소하고 속은 부드럽게 익힌다."

query_vector = embeddings.embed_query(question) #← 질문을 벡터화

print(f"[벡터화된 질문]\n- 임베딩개수:{len(query_vector)}개\n- 상위5개:{query_vector[:5]}") #← 벡터의 일부를 표시

document_1_vector = embeddings.embed_query(doc1) #← 문서 1의 벡터를 얻음
document_2_vector = embeddings.embed_query(doc2) #← 문서 2의 벡터를 얻음

cos_sim_1 = dot(query_vector, document_1_vector) / (norm(query_vector) * norm(document_1_vector)) #← 벡터의 유사도를 계산
print(f"문서 1과 질문의 유사도: {cos_sim_1:.3f}")
cos_sim_2 = dot(query_vector, document_2_vector) / (norm(query_vector) * norm(document_2_vector)) #← 벡터의 유사도를 계산
print(f"문서 2와 질문의 유사도: {cos_sim_2:.3f}")

[벡터화된 질문]
- 임베딩개수:1536개
- 상위5개:[-0.011089965681877331, -0.015286693322979184, 0.014691780295829944, -0.0299784745501317, -0.025658883109909277]
문서 1과 질문의 유사도: 0.933
문서 2와 질문의 유사도: 0.734


> 문서1의 유사도가 문서2보다 높으므로, 더 유사한 답변

- 벡터유사도 검색에서 RAG를 통합하는 구체적 절차
유저의 질문이 전송되었을 때, 원활하게 활용할 수 있는 RAG를 구성하려면 다음의 조건을 확인해야 한다.
1. 문서를 불러와서 (langchain.document_loaders)
    - PyMuPDFLoader, WebBaseLoader, csv_loader,...
2. 문서의 문장 구조를 적절히 분할 (langchain.text_splitter)
    - 어느 위치에서 쪼개서 읽는가에 따라 의미 달라짐. 보통 SpacyTextSplitter로 한국어의 주어, 서술어 등을 분석하여 적절한 위치에서 분할
3. 임베딩을 활용하여 문서 및 질문을 벡터화 (langchain.embeddings)
    - 위에서 OpenAIEmbeddings같이 문장을 비교하여 담아줄 그릇이 필요
4. 저장하였는가 (Vector Stores)
    - 그릇에 담은 벡터를 저장하는 DB 필요, Pinecorn, ChromaDB 등 많이씀 

## 2. PDF기반답변챗봇 만들기
> PDF에 포함된 정보를 기반으로 질문에 답하는 챗봇을 만들어보자

### PDF파일 불러오기

In [88]:
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("./asset/sample.pdf") #← sample.pdf 로드
documents = loader.load()

print(f"문서 개수: {len(documents)}") #← 문서 개수 확인
print(f"첫 번째 문서의 내용: {documents[0].page_content}") #← 첫 번째 문서의 내용을 확인
print(f"첫 번째 문서의 메타데이터: {documents[0].metadata}") #← 첫 번째 문서의 메타데이터를 확인

문서 개수: 12
첫 번째 문서의 내용: 하늘을 나는 자동차 관련 
법제도
주의】이 글은 가상의 비행 자동차를 대상으로 한 법률 자동 생성 예시입니다.

첫 번째 문서의 메타데이터: {'source': './asset/sample.pdf', 'file_path': './asset/sample.pdf', 'page': 0, 'total_pages': 12, 'format': 'PDF 1.7', 'title': '하늘을 나는 자동차 관련 법제도', 'author': '', 'subject': '', 'keywords': ', docId:825DD61FFAE9658C7293B36CB13F138C', 'creator': 'Microsoft Office Word', 'producer': 'Aspose.Words for .NET 23.11.0', 'creationDate': 'D:20231207125109Z', 'modDate': "D:20231211174122+09'00'", 'trapped': ''}


## 과제. 다른 파일도 불러와보세요
[한경컨센서스](https://consensus.hankyung.com/) 의 관심있는 보고서를 다운받아서 메타데이터를 확인 합니다. 아래 이어지는 내용도 실행한 데이터 기준으로 진행합니다

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("PDF파일주소") #← sample.pdf 로드
documents = loader.load()


# 첫번째 문서의 내용을 확인
# 첫 번째 문서의 메타데이터를 확인

> .document_loaders 로 가져오면 위와 같이 메타데이터 형태로 가져옴

### text_split

- PDF로 문장을 가져온 경우 RAG기법으로 처리하기에 너무 질어질 수도 있기에 text_splitter로 문맥을 유지하면서 문장을 적절히 분리
- spacy는 파이썬으로 개발된 NLP라이브러리로, 문장분할, 품사판단, 명사구 추출, 구분분석 가능
- spacy와 연동한 langchain의 SpacyTextSplitter로 적절히 분해하기
    + SpacyTextSplitter는 한국어의 주어, 서술어 등을 분석하여 적절한 위치에서 분할 할수 있다.

lang310가상환경에서 다음을 시행합니다 (한국어 모델 다운로드)
```Powershall
python -m spacy download ko_core_news_sm
```

In [67]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter  #← SpacyTextSplitter를 가져옴

loader = PyMuPDFLoader("./asset/sample.pdf") 
documents = loader.load()

text_splitter = SpacyTextSplitter(  #← SpacyTextSplitter를 초기화
    chunk_size=300,  #← 분할할 크기를 설정
    pipeline="ko_core_news_sm"  #← 분할에 사용할 언어 모델을 설정
)

splitted_documents = text_splitter.split_documents(documents) #← 문서를 분할

print(f"분할 전 문서 개수: {len(documents)}")
print(f"분할 후 문서 개수: {len(splitted_documents)}")

분할 전 문서 개수: 12
분할 후 문서 개수: 70


### Vector화
- OpenAI의 임베딩을 이용하려면 tiktoken 설치필요, VectorStore는 chromadb 설치
```Powershall
pip install tiktoken chromadb
```

In [95]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings  #← OpenAIEmbeddings 가져오기
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import Chroma  #← Chroma 가져오기
from dotenv import load_dotenv
import openai
import os

load_dotenv()
os.getenv("OPENAI_API_KEY")
loader = PyMuPDFLoader("./asset/sample.pdf")
documents = loader.load()

text_splitter = SpacyTextSplitter(
    chunk_size=300, 
    pipeline="ko_core_news_sm"
)

splitted_documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings( #← OpenAIEmbeddings를 초기화
    model="text-embedding-ada-002", #← 모델명을 지정
    api_key=os.getenv("OPENAI_API_KEY")
)

database = Chroma(  #← Chroma를 초기화
    persist_directory="./.tik",  #← 영속화 데이터 저장 위치 지정
    embedding_function=embeddings  #← 벡터화할 모델을 지정
)

database.add_documents(  #← 문서를 데이터베이스에 추가
    splitted_documents,  #← 추가할 문서 지정
)

print("데이터베이스 생성이 완료되었습니다.") #← 완료 알림

2024-05-05 20:39:23 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
데이터베이스 생성이 완료되었습니다.


> 각 질문과 유사도가 높은 부분을 추출해줌

In [96]:
res = database.similarity_search('비행 자동차의 최고 속도는?')
print(f"총 관련문서 : {len(res)}개")
print(res)

총 관련문서 : 4개
[Document(page_content='제2조(정의)\n이 법에서 "비행자동차"라 함은 지상 및 공중을 이동할 수 있는 능력을 가진 차량을 말한다. \n\n\n제3조(일반적 속도제한)\n\n\n1.\n도심에서 비행 자동차가 비행하는 경우 최대 속도는 시속 150km로 한다.\n\n\n2.\n\n도시 외의 지역에서 비행 자동차가 비행하는 경우 최대 속도는 시속 250km로 한다.\n\n\n3.\n\n특정 지역이나 시설 상공 또는 특정 비행 코스에서는 별도의 속도 제한이 설정될 수 있다.\n\n\n제4조 (특례 속도 제한)\n1.', metadata={'author': '', 'creationDate': 'D:20231207125109Z', 'creator': 'Microsoft Office Word', 'file_path': './asset/sample.pdf', 'format': 'PDF 1.7', 'keywords': ', docId:825DD61FFAE9658C7293B36CB13F138C', 'modDate': "D:20231211174122+09'00'", 'page': 3, 'producer': 'Aspose.Words for .NET 23.11.0', 'source': './asset/sample.pdf', 'subject': '', 'title': '하늘을 나는 자동차 관련 법제도', 'total_pages': 12, 'trapped': ''}), Document(page_content='비행 자동차 속도 제한법\n제1조(목적)\n이 법은 비행자동차의 비행 안전 및 일반 대중의 이익을 보장하기 위해 비행자동차의 비행 속도에 관한 \n기준을 규정하는 것을 목적으로 한다.\n\n\n제2조(정의)\n이 법에서 "비행자동차"라 함은 지상 및 공중을 이동할 수 있는 능력을 가진 차량을 말한다. \n\n\n제3조(일반적 속도제한)\n\n\n1.\n도심에서 비행 자동차가 비행하는 경우 최대 속도는 시속 150km로 한다.\n\n\n2.

In [97]:
'''
chroma로 저장한 벡터스토어는 Chroma메소드로 가져올수 있음
'''
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

db = Chroma(
    persist_directory="./.tik", 
    embedding_function=embeddings
)
retriever= db.as_retriever()
retriever.get_relevant_documents('비행 자동차 벌금은?')

2024-05-05 20:39:46 - Collection langchain is not created.


[Document(page_content='제6조 (자동비행 및 AI 위반 시 처벌)\n\n\n1.\n\n\n이 법을 위반하여 적절한 자동비행시스템 및 AI를 갖추지 않은 비행체를 운용한 자는 300만 원 이\n하의 벌금에 처한다.\n\n\n2.\n\n중대한 사고를 유발하거나 반복적으로 위반한 경우에는 제1항의 벌금 외에 5년 이하의 징역 \n또는 5억 원 이하의 벌금에 처한다.\n\n\n제7조(감독 및 지도)\n국가는 비행차량의 자동비행 및 AI의 준수 여부를 감독하고 필요한 경우 지도 또는 명령을 내릴 수 있다.\n\n\n제8조(법 시행일)\n이 법은 공포일로부터 6개월 후에 시행한다.', metadata={'author': '', 'creationDate': 'D:20231207125109Z', 'creator': 'Microsoft Office Word', 'file_path': './asset/sample.pdf', 'format': 'PDF 1.7', 'keywords': ', docId:825DD61FFAE9658C7293B36CB13F138C', 'modDate': "D:20231211174122+09'00'", 'page': 8, 'producer': 'Aspose.Words for .NET 23.11.0', 'source': './asset/sample.pdf', 'subject': '', 'title': '하늘을 나는 자동차 관련 법제도', 'total_pages': 12, 'trapped': ''}),
 Document(page_content='제7조(비\n행자동차 운전자격 위반 시 벌칙)\n1.\n\n이 법을 위반하여 적절한 비행자동차 운전 자격이 없는 자가 비행자동차를 운행한 경우 300만 원 \n이하의 벌금에 처한다.\n\n\n2.\n\n중대한 사고를 유발하거나 반복적으로 위반한 경우에는 제1항의 벌금 외에 5년 이하의 징역 \n또는 5억 원 이하의 벌금에 처한다.\n\n\n제8조(감독 및 지도)\n국가는 비행자동차의 운전자격 취득 

### 검색결과와 질문을 조합해서 질문에 답해보자

In [100]:
from langchain_community.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import openai
import os
input_message='비행 자동차의 최고 속도는?'

load_dotenv()
chat = ChatGroq(model="llama3-70b-8192",
               api_key = os.getenv("GROQ_API_KEY"))


# 프롬프트템플릿으로 LLM에 보낼 문서를 만들어줍니다.
prompt = PromptTemplate(template="""문장을 바탕으로 질문에 한국어로 답변

문장: 
{document}

질문: {query}
""", input_variables=["document", "query"])


# embedding 불러오고
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=os.getenv("OPENAI_API_KEY")
)

# VectorStore에서 문서를 loading하고
db = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

# Retriever를 만들어서 질문과 유사도 높은 문서 탐색
retriever = db.as_retriever()
documents = retriever.get_relevant_documents(input_message)

# 탐색한 문서를 str으로 붙여서 보낼 준비를 합니다.
documents_string = ""

for document in documents:
    documents_string += f"""
---------------------------
{document.page_content}
"""
# print(documents_string)

# 참고할 문서를 document에 붙여서 보내고, query를 던집니다.
# 위에서 prompt의 input_variables=["document", "query"] 로 셋팅했기에 각 변수 입력 가능
result = chat([
    HumanMessage(content=prompt.format(document=documents_string,
                                       query=input_message)) 
])
print(result.content)

2024-05-05 20:49:53 - Collection langchain is not created.
2024-05-05 20:49:54 - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
도심에서는 시속 150km, 도시 외의 지역에서는 시속 250km입니다.


### chainlit으로 채팅 서비스화
```
conda activate lang310
cd d:/drive/SelfStudy/dl.AI/ds4th_study/langchain/03_retrieval
chainlit run chat_2.py --port 8001
```


In [ ]:
'''
다음 코드를 빈문서에 넣고 chat_2.py로 저장하여 실행 
위의 코드와 동일하나, 함수를 채팅창의 각 메소드에 동기화 하는 코드 추가
'''
import chainlit as cl
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import openai
import os

load_dotenv()
os.getenv("OPENAI_API_KEY")

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

chat = ChatOpenAI(model="gpt-3.5-turbo")

prompt = PromptTemplate(template="""문장을 바탕으로 질문에 답하세요.

문장: 
{document}

질문: {query}
""", input_variables=["document", "query"])

database = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

@cl.on_chat_start # 채팅이 시작될때의 함수를 정의
async def on_chat_start():
    await cl.Message(content="준비되었습니다! 메시지를 입력하세요!").send()

@cl.on_message # 메시지 보낼 때 실행할 함수를 정의
async def on_message(input_message):
    print("입력된 메시지: " + input_message)
    documents = database.similarity_search(input_message)

    documents_string = ""

    for document in documents:
        documents_string += f"""
    ---------------------------
    {document.page_content}
    """

    result = chat([
        HumanMessage(content=prompt.format(document=documents_string,
                                           query=input_message)) #← input_message로 변경
    ])
    await cl.Message(content=result.content).send() #← 챗봇의 답변을 보냄

### 채팅시작 시 파일업로드

In [ ]:
import os
import chainlit as cl
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import openai
import os

load_dotenv()
os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

chat = ChatOpenAI(model="gpt-3.5-turbo")

prompt = PromptTemplate(template="""문장을 기반으로 질문에 답하세요. 

문장: 
{document}

질문: {query}
""", input_variables=["document", "query"])

text_splitter = SpacyTextSplitter(chunk_size=300, pipeline="ko_core_news_sm")

@cl.on_chat_start
async def on_chat_start():
    files = None #← 파일이 선택되어 있는지 확인하는 변수

    while files is None: #← 파일이 선택될 때까지 반복
        files = await cl.AskFileMessage(
            max_size_mb=20,
            content="PDF를 선택해 주세요",
            accept=["application/pdf"],
            raise_on_timeout=False,
        ).send()
    file = files[0]

    if not os.path.exists("tmp"): #← tmp 디렉터리가 존재하는지 확인
        os.mkdir("tmp") #← 존재하지 않으면 생성
    with open(f"tmp/{file.name}", "wb") as f: #← PDF 파일을 저장
        f.write(file.content) #← 파일 내용을 작성

    documents = PyMuPDFLoader(f"tmp/{file.name}").load() #← 저장한 PDF 파일을 로드
    splitted_documents = text_splitter.split_documents(documents) #← 문서를 분할

    database = Chroma( #← 데이터베이스 초기화
        embedding_function=embeddings,
        # 이번에는 persist_directory를 지정하지 않음으로써 데이터베이스 영속화를 하지 않음
    )

    database.add_documents(splitted_documents) #← 문서를 데이터베이스에 추가

    cl.user_session.set(  #← 데이터베이스를 세션에 저장
        "database",  #← 세션에 저장할 이름
        database  #← 세션에 저장할 값
    )

    await cl.Message(content=f"`{file.name}` 로딩이 완료되었습니다. 질문을 입력하세요.").send() #← 불러오기 완료를 알림

@cl.on_message
async def on_message(input_message):
    print("입력된 메시지: " + input_message)

    database = cl.user_session.get("database") #← 세션에서 데이터베이스를 가져옴

    documents = database.similarity_search(input_message)

    documents_string = ""

    for document in documents:
        documents_string += f"""
    ---------------------------
    {document.page_content}
    """
    
    result = chat([
        HumanMessage(content=prompt.format(document=documents_string,
                                           query=input_message)) #← input_message로 변경
    ])
    await cl.Message(content=result.content).send()

## 3. RetrievalQA
- RAG 기능을 활용한 QA 시스템 개발을 보다 쉽고 다기능으로 만들어줌
- 검색, 프롬프트구축, 언어모델호출처리 구현을 단순화하여 코드호출을 줄임

In [102]:
from langchain.chat_models import ChatOpenAI  #← ChatOpenAI 가져오기
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate  #← PromptTemplate 가져오기
from langchain.schema import HumanMessage  #← HumanMessage 가져오기
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import openai
import os

load_dotenv()
os.getenv("OPENAI_API_KEY")

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

database = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

query = "비행 자동차의 최고 속도는?"

documents = database.similarity_search(query)

documents_string = "" #← 문서 내용을 저장할 변수를 초기화

for document in documents:
    documents_string += f"""
---------------------------
{document.page_content}
""" #← 문서 내용을 추가

prompt = PromptTemplate( #← PromptTemplate를 초기화
    template="""문장을 바탕으로 질문에 답하세요.

문장: 
{document}

질문: {query}
""",
    input_variables=["document","query"] #← 입력 변수를 지정
)

chat = ChatOpenAI( #← ChatOpenAI를 초기화
    model="gpt-3.5-turbo"
)

result = chat([
    HumanMessage(content=prompt.format(document=documents_string, query=query))
])

print(result.content)


2024-05-05 21:55:39 - Collection langchain is not created.
도심에서는 최대 속도가 시속 150km이고, 도시 외의 지역에서는 최대 속도가 시속 250km입니다.


In [110]:
from langchain.chains import RetrievalQA  #← RetrievalQA를 가져오기
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

chat = ChatOpenAI(model="gpt-3.5-turbo")

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

database = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

retriever = database.as_retriever() #← 데이터베이스를 Retriever로 변환

# qa안에 llm, retriever를 한번에 설정하여 document, query를 별도 입력할 필요가 없음 (암묵적 수행)
qa = RetrievalQA.from_llm(  #← RetrievalQA를 초기화
    llm=chat,  #← Chat models를 지정
    retriever=retriever,  #← Retriever를 지정
    return_source_documents=True,  #← 응답에 원본 문서를 포함할지를 지정
)

result = qa("비행 자동차의 최고 속도를 알려주세요")

print(result["result"]) #← 응답을 표시

# print(result["source_documents"]) #← 원본 문서를 표시

2024-05-05 22:01:58 - Collection langchain is not created.
도심에서 비행 자동차가 비행하는 경우 최대 속도는 시속 150km이며, 도시 외의 지역에서 비행 자동차가 비행하는 경우 최대 속도는 시속 250km입니다.


## 4.준비된 Retrievers 활용
> pdf문서가 없어도 wikipedia를 활용하여 답변을 생성할 수 있다. WikipediaRetriever를 사용
- 키워드를 명사중심으로 검색해야 명확하게 검색이 됨ㅁ

In [131]:
'''
pip install wikipedia
'''
from langchain.retrievers import WikipediaRetriever

retriever = WikipediaRetriever(  #← WikipediaRetriever를 초기화
    lang="ko",  #← Wikipedia의 언어를 지정
)
documents = retriever.get_relevant_documents( #← Wikipedia에서 관련 문서를 가져옴
    "피카츄" #← 검색할 키워드를 지정
)

print(f"검색 결과: {len(documents)}건") #← 검색 결과 건수를 표시

for document in documents:
    print("---------------검색한 메타데이터---------------")
    print(document.metadata) #← 메타데이터를 표시
    print("---------------검색한 텍스트---------------")
    print(document.page_content[:100]) #← 텍스트의 첫 100글자를 표시

검색 결과: 3건
---------------검색한 메타데이터---------------
{'title': '피카츄', 'summary': '피카츄(일본어: ピカチュウ 피카추[*] 문화어: 삐까쮸)는 《포켓몬스터》에 등장하는 가상의 생명체이다. 애니메이션과 비디오 게임에서는 오타니 이쿠에가 그 목소리를 맡고 있다. 귀여운 전기 포켓몬을 그려달라는 스기모리 켄의 주문을 받아 니시다 아츠코가 디자인하였다. 게임 프리크와 닌텐도가 만든 1996년 일본 비디오 게임 《포켓몬스터 레드·그린》에 처음 등장했으며 1998년 《포켓몬스터 레드·블루》로 미국에 출시되었다. 피카츄는 전기 능력을 가진 노란색 쥐처럼 생긴 생물이다. 피카츄는 포켓몬 프랜차이즈의 주요 캐릭터이며, 마스코트이자 닌텐도의 주요 마스코트 역할을 한다.\n피카츄는 가장 인기 있고 잘 알려진 1세대 포켓몬인데, 포켓몬스터 TV 애니메이션에서 주인공 지우의 첫 포켓몬이자 파트너로 등장하기 때문이다. 1998년부터 지금까지 포켓몬스터의 애니메이션 주인공 지우와 함께 모험하고 있다. 피카츄는 대부분의 목소리 출연을 오타니 이쿠에가 담당했지만, 실사 애니메이션 영화 《명탐정 피카츄》에서 라이언 레이놀즈가 담당했다. 피카츄는 특히 귀여움으로 호평받으며 일본 대중문화의 아이콘으로 여겨지고 있다.', 'source': 'https://ko.wikipedia.org/wiki/%ED%94%BC%EC%B9%B4%EC%B8%84'}
---------------검색한 텍스트---------------
피카츄(일본어: ピカチュウ 피카추[*] 문화어: 삐까쮸)는 《포켓몬스터》에 등장하는 가상의 생명체이다. 애니메이션과 비디오 게임에서는 오타니 이쿠에가 그 목소리를 맡고 있다. 귀여
---------------검색한 메타데이터---------------
{'title': '포켓몬스터 피카츄', 'summary': '《포켓몬스터 피카츄》(일본어: ポケットモンスター ピカチュウ )는 1998년 출시 된, 포켓몬스터 청의 후속작이다. 애니메

In [139]:
from langchain.chains import RetrievalQA  #← RetrievalQA를 가져오기
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
input_text = "피카츄는 언제 출시 되었어?"
input_ = '피카츄'
chat = ChatOpenAI(model="gpt-3.5-turbo")

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
retriever = WikipediaRetriever(  #← WikipediaRetriever를 초기화
    lang="ko",  #← Wikipedia의 언어를 지정
)
documents = retriever.get_relevant_documents( #← Wikipedia에서 관련 문서를 가져옴
    input_ #← 검색할 키워드를 지정
)

retriever = database.as_retriever() #← 데이터베이스를 Retriever로 변환

# qa안에 llm, retriever를 한번에 설정하여 document, query를 별도 입력할 필요가 없음 (암묵적 수행)
qa = RetrievalQA.from_llm(  #← RetrievalQA를 초기화
    llm=chat,  #← Chat models를 지정
    retriever=retriever,  #← Retriever를 지정
    return_source_documents=True,  #← 응답에 원본 문서를 포함할지를 지정
)

result = qa(input_text)

print(result["result"]) #← 응답을 표시

# print(result["source_documents"]) #← 원본 문서를 표시

제가 알기로는 피카츄는 1996년에 처음 출시되었습니다.


### LLMChain, RePhraseQueryRetriever 활용하여 키워드 추출

In [140]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever, RePhraseQueryRetriever #← RePhraseQueryRetriever를 가져오기
from langchain import LLMChain
from langchain.prompts import PromptTemplate

retriever = WikipediaRetriever( 
    lang="ko", 
    doc_content_chars_max=500 
)

llm_chain = LLMChain( #← LLMChain을 초기화
    llm = ChatOpenAI( #← ChatOpenAI를 지정
        temperature = 0
    ), 
    prompt= PromptTemplate( #← PromptTemplate을 지정
        input_variables=["question"],
        template="""아래 질문에서 Wikipedia에서 검색할 키워드를 추출해 주세요.
질문: {question}
"""
))

re_phrase_query_retriever = RePhraseQueryRetriever( #← RePhraseQueryRetriever를 초기화
    llm_chain=llm_chain, #← LLMChain을 지정
    retriever=retriever, #← WikipediaRetriever를 지정
)

documents = re_phrase_query_retriever.get_relevant_documents("나는 라면을 좋아합니다. 그런데 소주란 무엇인가요?")

print(documents)

2024-05-05 22:24:45 - Re-phrased question: 키워드: 라면, 소주
[Document(page_content='보쌈(영어: bossam)은 쌈의 일종으로서 돼지고기를 삶은 수육(삶아서 물기를 뺀 고기)에 상추나 절인 배춧잎으로 쌈을 해서 먹는 한국 요리다. 대개 소, 중, 대 등으로 양을 구분해 판매한다. 한국인이 소주를 마실 때 많이 찾는 요리 중 하나다. 본래 고기를 삶아 피와 기름을 빼서 먹는 것이 보쌈고기의 가장 큰 특징과 일부 체인점에서는 기름맛을 내기 위해 삶지 않고 찌기도 한다. 참고로 보쌈의 보는 한자어로 보자기(포대기)를 의미한다.\n돼지고기를 삶을 때에는 생강을 이용해서 고기 자체의 비린내를 제거하는데 맛을 좋게 하기 위해 여러 가지 한약재, 무, 양파 등을 같이 넣어 삶기도 한다.\n 보쌈을 먹을 때에는 쌈장을 찍어서 상추와 깻잎에 싸 먹는다. 새우젓을 함께 찍어 먹기도 하며, 무말랭이 따위가 함께 나오는 것도 흔한 편이다. 굴을 함께 낸 굴보쌈도 있다.\n대한민국에는 많은 보쌈 요리 체인점이 있으며, 서울특별시 종로구 종로3가 인근에는 보쌈 골목이 있다.\n\n\n== 각주 ==', metadata={'title': '보쌈', 'summary': '보쌈(영어: bossam)은 쌈의 일종으로서 돼지고기를 삶은 수육(삶아서 물기를 뺀 고기)에 상추나 절인 배춧잎으로 쌈을 해서 먹는 한국 요리다. 대개 소, 중, 대 등으로 양을 구분해 판매한다. 한국인이 소주를 마실 때 많이 찾는 요리 중 하나다. 본래 고기를 삶아 피와 기름을 빼서 먹는 것이 보쌈고기의 가장 큰 특징과 일부 체인점에서는 기름맛을 내기 위해 삶지 않고 찌기도 한다. 참고로 보쌈의 보는 한자어로 보자기(포대기)를 의미한다.\n돼지고기를 삶을 때에는 생강을 이용해서 고기 자체의 비린내를 제거하는데 맛을 좋게 하기 위해 여러 가지 한약재, 무, 양파 등을 같이 넣어 삶기도 한다.\n 보쌈을 먹을 때에는 쌈장을 찍어서 상추와 깻잎에 싸 먹는다. 새우젓을 

### 과제> 질문으로 키워드 불러오는 것을 자동화 하였으니, 다시 자동 검색으로 적용해봅시다.

In [146]:
from langchain.chains import RetrievalQA  #← RetrievalQA를 가져오기
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain import LLMChain
from langchain.retrievers import WikipediaRetriever, RePhraseQueryRetriever #← RePhraseQueryRetriever를 가져오기

input_text = "피카츄는 언제 출시 되었어?"
chat = ChatOpenAI(model="gpt-3.5-turbo")

retriever = WikipediaRetriever( 
    lang="ko", 
    doc_content_chars_max=500 
)


#### 답안

In [ ]:
from langchain.chains import RetrievalQA  #← RetrievalQA를 가져오기
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain import LLMChain
from langchain.retrievers import WikipediaRetriever, RePhraseQueryRetriever #← RePhraseQueryRetriever를 가져오기

input_text = "피카츄는 언제 출시 되었어?"
chat = ChatOpenAI(model="gpt-3.5-turbo")

retriever = WikipediaRetriever( 
    lang="ko", 
    doc_content_chars_max=500 
)

llm_chain = LLMChain( #← LLMChain을 초기화
    llm = ChatOpenAI( #← ChatOpenAI를 지정
        temperature = 0), 
    prompt= PromptTemplate( #← PromptTemplate을 지정
        input_variables=["question"],
        template="""아래 질문에서 Wikipedia에서 검색할 키워드를 추출해 주세요.
질문: {question}
"""
))

re_phrase_query_retriever = RePhraseQueryRetriever( #← RePhraseQueryRetriever를 초기화
    llm_chain=llm_chain, #← LLMChain을 지정
    retriever=retriever, #← WikipediaRetriever를 지정
)

qa = RetrievalQA.from_llm(  #← RetrievalQA를 초기화
    llm=chat,  #← Chat models를 지정
    retriever=re_phrase_query_retriever,  #← Retriever를 지정
    return_source_documents=True,  #← 응답에 원본 문서를 포함할지를 지정
)

result = qa(input_text)

print(result["result"]) #← 응답을 표시

# print(result["source_documents"]) #← 원본 문서를 표시